# Test Order Service and Models
- This notebook allows experimentation and testing  of the Order components 

<span style="color:red; font-weight:bold">Note:</span> Remember to Restart the Notebook **if you have changed code**.<br>The notebook does not auto reload.


In [1]:
# System level imports
import sys
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
TEST_DEV_PRJ_PATH = os.environ.get("TEST_DEV_PRJ_PATH")

# ##### SET SYS PATH TO WHERE THE CODE IS. #####
# Note: Putting our code first in the sys path will make sure it gets picked up
sys.path.insert(0, f"{TEST_DEV_PRJ_PATH}/backend/src")
print(sys.path)

['/Users/dan/Code/Python/Ken/routes-splitter/backend/src', '/Users/dan/miniforge3/envs/py3.13/lib/python313.zip', '/Users/dan/miniforge3/envs/py3.13/lib/python3.13', '/Users/dan/miniforge3/envs/py3.13/lib/python3.13/lib-dynload', '', '/Users/dan/Code/Python/Ken/routes-splitter/.venv/lib/python3.13/site-packages']


In [2]:
# What order data are we looking at?
import os
print(os.getcwd())
relative_path = "../../data/daily_orders.xlsx"
full_path = os.path.abspath(relative_path)
print(full_path)

/Users/dan/Code/Python/Ken/routes-splitter/backend/notebooks
/Users/dan/Code/Python/Ken/routes-splitter/data/daily_orders.xlsx


In [3]:
# Get the services that we will use
from wulfs_routing_api.services.order_services import OrderService
from wulfs_routing_api.models.orders.supabase_order import OrderModel
from wulfs_routing_api.models.customers.supabase_customer import SupabaseCustomer
from wulfs_routing_api.services.customer_service import CustomerService

customer_service = CustomerService(SupabaseCustomer())
order_service = OrderService(OrderModel())

✅ Supabase client initialized.


In [4]:
# Test the load_orders
from wulfs_routing_api.utils.data_io_utils import read_table

daily_orders_path = "../../data/daily_orders.xlsx" 
daily_orders_df = read_table(daily_orders_path)
daily_orders_df

,Customer Name
0,89 Charles
1,Bar Volpe
2,Fox & The Knife
3,Dryft Revere
4,Mila's Revere
...,...
104,Cosmica
105,The Beehive
106,"Wilson Farm, Inc."
107,Zuma-Boston


In [5]:
# Get the customer master 
# Note: This could be mocked later as we do not actually want to test customer
customer_master_df = customer_service.load_customer_master_data()
customer_master_df

,customer_id,name_key,customer_name,address,city,state,zip,lat,lon
0,1,1928 beacon hill llc,1928 Beacon Hill LLC,97 Mt. Vernon St,Boston,MA,2108,42.358085,-71.069732
1,2,ac kitchen,AC Kitchen,161 Merrimac St,Woburn,MA,1801,42.514958,-71.148602
2,3,bar volpe,Bar Volpe,170 W Broadway,Boston,MA,2127,42.340412,-71.053037
3,4,fox the knife,Fox & The Knife,28 West Broadway,South Boston,MA,2127,42.340672,-71.053535
4,5,alba restaurant,Alba Restaurant,1486 Hancock Street,Quincy,MA,2169,42.248760,-71.001785
...,...,...,...,...,...,...,...,...,...
488,489,wequassett resort and golf club,Wequassett Resort and golf club,2173 MA-28,Harwich,MA,2645,41.684283,-70.011038
489,490,westin boston sauciety restaurant,Westin Boston Sauciety Restaurant,425 Summer St,Boston,MA,2210,42.346186,-71.042738
490,491,the beehive co spybar,The Beehive CO SpyBar,541 Tremont,St Boston,MA,2116,42.344373,-71.071381
491,492,leybi l lopez sales,Leybi L Lopez Sales,8 Seafood Way,Boston,MA,2210,42.349018,-71.031133


In [6]:
orders_w_customers_df, missing_customers_df = order_service.customer_details_for_orders(daily_orders_df, customer_master_df)
orders_w_customers_df.head()


,customer_name_order,order_id,notes,name_key,customer_id,customer_name,address,city,state,zip,lat,lon
0,Bar Volpe,,,bar volpe,3.0,Bar Volpe,170 W Broadway,Boston,MA,2127,42.340412,-71.053037
1,Fox & The Knife,,,fox the knife,4.0,Fox & The Knife,28 West Broadway,South Boston,MA,2127,42.340672,-71.053535
2,Dryft Revere,,,dryft revere,7.0,Dryft Revere,500 Ocean Ave,Revere,MA,2151,42.414102,-70.990817
3,Bar Mezzana,,,bar mezzana,13.0,Bar Mezzana,360 Harrison Avenue,Boston,MA,2118,42.344587,-71.063512
4,Black Lamb,,,black lamb,14.0,Black Lamb,571 Tremont Street,Boston,MA,2118,42.343603,-71.072514


In [7]:
print(f"Number of Missing Customer Details for Daily Orders {len(missing_customers_df)}")
missing_customers_df.head()

Number of Missing Customer Details for Daily Orders 40


,customer_name_order,order_id,notes,name_key,customer_id,customer_name,address,city,state,zip,lat,lon
0,89 Charles,,,89 charles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mila's Revere,,,milas revere,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Bambara,,,bambara,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Shore Leave,,,shore leave,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,Bubble Bath Back Bay,,,bubble bath back bay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Save this result to be used by the VRP Solver
# Lets put it in ../data for now
orders_w_customers_df.to_csv("../../data/orders_w_customers.csv",index=False)